# Filter the points outside of the ROI

In [3]:
import os
import pandas as pd
import numpy as np
from pathlib import Path

## Constants

In [5]:
DATA_DIR_ROOT = '../stationary_data'

In [6]:
def point_is_in_an_roi_box(point, roi_box_list):
    # Extract the point coordinates
    x, y, z = point
    
    for bbox in roi_box_list:
        # Extract the box parameters
        # dz, dy, dx, cx, cy, cz, yaw = bbox
        dx, dy, dz, cx, cy, cz, yaw = bbox
        
        
        # Create the rotation matrix for the yaw angle
        cos_yaw = np.cos(-yaw)  # Negative for the inverse rotation
        sin_yaw = np.sin(-yaw)
        
        rotation_matrix = np.array([
            [cos_yaw, -sin_yaw, 0],
            [sin_yaw,  cos_yaw, 0],
            [0,       0,       1]
        ])
        
        # Translate the point to the box's coordinate frame
        translated_point = np.array([x - cx, y - cy, z - cz])
        
        # Rotate the point to align with the box
        rotated_point = rotation_matrix.dot(translated_point)
        
        # Check if the point is within the box dimensions
        half_lengths = [dz / 2, dy / 2, dx / 2]
        in_box = all([
            -half_lengths[i] <= rotated_point[i] <= half_lengths[i] for i in range(3)
        ])
    
        if in_box:
            return True
    
    return False

In [7]:
def get_roi_boxes(label_file):
    bboxes = []
    with open(label_file, 'r') as file:
        for line in file:
            parts = line.strip().split()
            bbox = [float(value) for value in parts[8:15]]  # Extract the bounding box dimensions and location
            bboxes.append(bbox)
    return np.array(bboxes)
    # dz, dy, dx, x, y, z, yaw = bbox

In [8]:
def convert_to_dataframe(bin_path):
    pre_filtered_data = np.fromfile(bin_path, dtype=np.float32).reshape(-1, 4) 
    columns = ['x', 'y', 'z', 'intensity']
    df = pd.DataFrame(pre_filtered_data, columns=columns)
    return df

In [9]:
def roi_filter_points(input_file, label_file):
    # Get dataframe from input file
    input_df = convert_to_dataframe(input_file)
    list_roi_boxes = get_roi_boxes(label_file)
    print(list_roi_boxes)
    
    # Initialize a list to store rows that meet the criteria
    filtered_data = []

    # Iterate through each row in the input DataFrame
    for index, row in input_df.iterrows():
    # for index, row in input_df.iloc[:3].iterrows():
        point = (row['x'], row['y'], row['z'])
        # Check if the point is inside any of the ROI boxes
        if point_is_in_an_roi_box(point, list_roi_boxes):
            filtered_data.append(row)
        

    # Create a DataFrame from the filtered data
    filtered_df = pd.DataFrame(filtered_data)
    return filtered_df

In [10]:
def save_as_binary(df, bin_path):
    # Ensure the DataFrame is in the correct order and data type
    data = df[['x', 'y', 'z', 'intensity']].astype(np.float32).values
    
    # Write the data to a binary file
    data.tofile(bin_path)

In [11]:
def process_directory(dir):
    print(dir)
    # Make new folder for filtered frames
    new_save_location = Path(dir, 'roi_filtered_points')
    new_save_location.mkdir(exist_ok=True)

    # Original frame path
    lidar_dir = Path(dir, 'velodyne_points')
    label_dir = Path(dir, 'labels')
    
    # Get just the file names
    files = [f for f in os.listdir(lidar_dir) if f.endswith('.bin')]
    # For each file
    # for filename in files[:3]:
    for filename in files[:3]:
        print('.', end='')
        # Append file name to location
        from_file = Path(lidar_dir, filename)
        label_file = Path(label_dir, filename[:-4] + '.txt')

        # Filter file
        # SEND BOTH BIN AND LABELS
        roi_filtered_df = roi_filter_points(from_file, label_file)

        # APPEND FILE NAME TO NEW LOCATION
        to_file = Path(new_save_location, filename)

        # CONVERT BACK TO BINARY and save
        save_as_binary(roi_filtered_df, to_file)

    print()
    print()

In [12]:
def filter_by_roi():
    # For each folder in the stationary data directory
    p = Path(DATA_DIR_ROOT)
    
    # For each sequence (folder) in the stationary data
    for dir in p.iterdir(): 
        if dir.is_dir():
            process_directory(dir)
            

In [13]:
filter_by_roi()

..\stationary_data\2011_09_26_drive_0017_sync_0_to_113
.[[ 1.73  1.83  4.16 13.06 -4.   -1.6  -1.75]
 [ 1.55  1.66  3.37 16.68 14.34 -1.45 -1.73]]
.[[ 1.73  1.83  4.16 12.87 -4.92 -1.61 -1.75]
 [ 1.55  1.66  3.37 16.5  13.24 -1.46 -1.73]]
.[[ 1.73  1.83  4.16 12.65 -6.   -1.62 -1.75]
 [ 1.55  1.66  3.37 16.31 12.14 -1.47 -1.73]]


..\stationary_data\2011_09_26_drive_0018_sync_0_to_178
.[[ 1.42  1.47  3.52 12.09  3.26 -1.59 -3.93]]
.[[ 1.42  1.47  3.52 11.39  3.9  -1.56 -3.87]]
.[[ 1.42  1.47  3.52 10.6   4.53 -1.57 -3.81]]


..\stationary_data\2011_09_26_drive_0051_sync_210_to_210
.[[ 1.35  1.6   3.86 40.11 -3.15 -1.11 -0.3 ]
 [ 2.08  2.05  4.96 42.94 -7.19 -0.95 -0.23]
 [ 1.32  1.48  4.44 15.72  4.31 -1.45 -0.33]
 [ 2.3   1.87  5.7  33.88  8.14 -1.47 -3.3 ]
 [ 1.44  1.54  3.15 22.73 10.21 -1.68 -3.37]
 [ 4.02  2.6  16.79 60.72  5.25 -1.4  -3.18]
 [ 1.49  1.56  3.65 55.56 -6.92 -0.98 -0.23]]


..\stationary_data\2011_09_26_drive_0051_sync_224_to_360
.[[  1.35   1.6    3.86  51.9   -6.2

KeyError: "None of [Index(['x', 'y', 'z', 'intensity'], dtype='object')] are in the [columns]"